In [2]:
%matplotlib inline
from os import listdir, path
from requests import get
import urllib3
from lxml import etree
import pandas as pd
import warnings
from tqdm import tqdm
import itertools
warnings.filterwarnings('ignore')
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [4]:
for year in tqdm(range(2011, 2018)):
    stop = False
    i=0
    while stop == False:
        i+=1
        for j in range(1, 200):
            file = "h-tk-%s-%s-%s" %(str(year)+str(year+1), str(i), str(j))
            if not path.isfile('HandelingenTK/%s.xml' %file):
                url = "https://zoek.officielebekendmakingen.nl/%s" %file
                resp = get(url + ".xml", verify=False, allow_redirects=False)
                if resp.status_code == 200:
                    hand_file = open('HandelingenTK/%s.xml' %file, 'wb')
                    hand_file.write(resp.content)
                    hand_file.close()
                    resp = get(url+"/metadata.xml", verify=False, \
                                                        allow_redirects=False)
                    meta_file = open('HandelingenTKmeta/%smetadata.xml' %file, 'wb')
                    meta_file.write(resp.content)
                    meta_file.close()    
                elif j==1:
                    stop = True
                    break
                else:
                    break

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [06:34<00:00, 56.29s/it]


In [3]:
enddict = {}
i = 0
for file in listdir('HandelingenTK/'):
    context = etree.iterparse('HandelingenTK/' + file,  tag='spreekbeurt')
    for _, elem in context:
        if elem.get('nieuw') == 'ja':
            spreekbeurt = "ja"
        else:
            spreekbeurt = "nee"
        surname = elem.findtext('spreker/naam/achternaam')
        party = elem.findtext('spreker/politiek')
        text = ' '.join(elem.find('tekst').itertext())
        enddict[i] = [surname, party, text, file,spreekbeurt]
        del surname, party, text, spreekbeurt
        i += 1
df = pd.DataFrame.from_dict(enddict, orient='index')
df = df.rename(columns={0:'achternaam', 1:'partij', 2:'tekst', 3:'file', 4:'spreekbeurt'})

In [4]:
enddict = {}
for file in listdir('HandelingenTKmeta'):
    tags = []
    datum = None
    context = etree.iterparse('HandelingenTKmeta/' + file,  tag='metadata')
    for _, elem in context:
        if elem.get('name') == "OVERHEIDop.datumVergadering":
            datum = elem.get('content')
        elif elem.get('name') == "OVERHEID.category":
            tags.append(elem.get('content'))
        enddict[file[:-12]+'.xml'] = [datum, tags]
df2 = pd.DataFrame.from_dict(enddict, orient='index')
df2 = df2.rename(columns={0:'datum', 1:'tags'})

In [5]:
combineddf = df.join(df2, on='file')
combineddf = combineddf.dropna()
#https://stackoverflow.com/questions/29370057/select-dataframe-rows-between-two-dates
combineddf['datum'] = pd.to_datetime(combineddf['datum'])
mask = (combineddf['datum'] > '2012-11-05') & (combineddf['datum'] <= '2017-03-23')
combineddf = combineddf.loc[mask]
combineddf = combineddf.loc[combineddf['spreekbeurt'] == 'ja']
combineddf = combineddf[combineddf['partij'].isin(['50PLUS', 'CDA','ChristenUnie','D66','GroenLinks','PVV','PvdA','PvdD','SGP','SP','VVD'])]
combineddf.to_csv("Handelingen.csv")
combineddf.head(5)

,achternaam,partij,tekst,file,spreekbeurt,datum,tags
107136,Wilders,PVV,\n \n Mevrouw de voorzit...,h-tk-20122013-100-3.xml,ja,2013-06-26,"[Bestuur | Parlement, Financiën | Begroting]"
107137,Roemer,SP,\n \n Voorzitter. Vorige...,h-tk-20122013-100-3.xml,ja,2013-06-26,"[Bestuur | Parlement, Financiën | Begroting]"
107138,Pechtold,D66,\n \n Voorzitter. Twee m...,h-tk-20122013-100-3.xml,ja,2013-06-26,"[Bestuur | Parlement, Financiën | Begroting]"
107139,Van Haersma Buma,CDA,\n \n Voorzitter. Alleen...,h-tk-20122013-100-3.xml,ja,2013-06-26,"[Bestuur | Parlement, Financiën | Begroting]"
107148,Thieme,PvdD,\n \n Voorzitter. We zij...,h-tk-20122013-100-3.xml,ja,2013-06-26,"[Bestuur | Parlement, Financiën | Begroting]"
